In [2]:
# 필요한 라이브러리 임포트
from binance.client import Client
from binance.enums import *
import pandas as pd
from datetime import datetime , timedelta
import time

import pandas as pd
import numpy as np
import torch
import pyarrow

In [3]:


# API 키 설정
api_key = ''
api_secret = ''


# 바이낸스 클라이언트 초기화
client = Client(api_key, api_secret)


In [4]:
class Spot:
    def __init__(self):
        self.spot = None
        self.curent_symbols = self.get_all_spot_symbols()
    
    def get_all_spot_symbols(self):
        """
            거래 가능한 모든 현물 심볼 목록 반환
        """
        try:
            exchange_info = client.get_exchange_info()
            return [symbol['symbol'] for symbol in exchange_info['symbols'] if symbol['status'] == 'TRADING' and symbol['quoteAsset'] == 'USDT']
        except Exception as e:
            print(f"심볼 목록 조회 에러: {e}")
            return None
    
    def get_spot_klines(self,symbol='BTCUSDT', interval='1d', limit=500):
        """
        현물 캔들스틱 데이터 가져오기
        :param symbol: 거래쌍 (예: 'BTCUSDT')
        :param interval: 시간간격 (1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M)
        :param limit: 가져올 캔들 수
        :return: DataFrame
        """
        try:
            klines = client.get_historical_klines(symbol=symbol, interval=interval, limit=limit)
            
            # DataFrame 생성
            df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 
                                            'volume', 'close_time', 'quote_asset_volume', 'number_of_trades',
                                            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
            
            # 데이터 타입 변환
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            for col in ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume']:
                df[col] = df[col].astype(float)
                
            return df[['timestamp', 'close']]
        
        except Exception as e:
            print(f"에러 발생: {e}")
            return None
    
    def get_spot_historical_klines(self, symbol='BTCUSDT', interval='1d', start_str='2017-01-01', end_str=None):
        """
        특정 기간의 현물 캔들스틱 데이터 가져오기
        :param symbol: 거래쌍 또는 'all'
        :param interval: 시간간격
        :param start_str: 시작 날짜 (YYYY-MM-DD)
        :param end_str: 종료 날짜 (YYYY-MM-DD)
        :return: Dictionary of DataFrames (symbol을 'all'로 지정한 경우) 또는 단일 DataFrame
        """
        try:
            # 날짜를 타임스탬프로 변환
            start_ts = int(pd.Timestamp(start_str).timestamp() * 1000)

            if end_str == None:
                end_ts = datetime.now().strftime('%Y-%m-%d')
                end_ts = int(pd.Timestamp(end_ts).timestamp() * 1000)
            else:
                end_ts = int(pd.Timestamp(end_str).timestamp() * 1000)

            if symbol.lower() == 'all':
                symbols = self.get_all_spot_symbols()
                if not symbols:
                    return None
                
                result = {}
                for sym in symbols:
                    print(f"{sym} 데이터 수집 중...")
                    klines = []
                    while start_ts < end_ts:
                        temp_data = client.get_historical_klines(
                            symbol=sym,
                            interval=interval,
                            start_str=str(start_ts),
                            end_str=str(end_ts),
                            limit=1000
                        )
                        if not temp_data:
                            break
                        klines.extend(temp_data)
                        start_ts = temp_data[-1][0] + 1  # 다음 시작점을 마지막 데이터의 다음 시점으로 설정
                        time.sleep(0.1)  # API 호출 제한 방지
                    
                    if klines:
                        df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 
                                                        'volume', 'close_time', 'quote_volume', 'trades',
                                                        'taker_base', 'taker_quote', 'ignore'])
                        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                        for col in ['open', 'high', 'low', 'close', 'volume', 'quote_volume']:
                            df[col] = df[col].astype(float)
                        result[sym] = df[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'quote_volume']]
                    time.sleep(0.5)
                return result
            
            else:
                klines = []
                while start_ts < end_ts:
                    temp_data = client.get_historical_klines(
                        symbol=symbol,
                        interval=interval,
                        start_str=str(start_ts),
                        end_str=str(end_ts),
                        limit=1000
                    )
                    if not temp_data:
                        break
                    klines.extend(temp_data)
                    start_ts = temp_data[-1][0] + 1  # 다음 시작점을 마지막 데이터의 다음 시점으로 설정
                    time.sleep(0.1)  # API 호출 제한 방지
                
                df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 
                                                'volume', 'close_time', 'quote_volume', 'trades',
                                                'taker_base', 'taker_quote', 'ignore'])
                
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                for col in ['open', 'high', 'low', 'close', 'volume', 'quote_volume']:
                    df[col] = df[col].astype(float)
                    
                return df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
        
        except Exception as e:
            print(f"에러 발생: {e}")
            return None
    
    def fetch_and_merge_close_prices(self, interval='1d', limit=2000):
        """
        모든 심볼의 종가 데이터를 불러와 병합한 DataFrame 반환
        :param interval: 시간간격
        :param limit: 가져올 캔들 수
        :return: 병합된 DataFrame
        """
        symbols = self.get_all_spot_symbols()
        if not symbols:
            return None
        
        merged_df = pd.DataFrame()

        for i, symbol in enumerate(symbols):
            print(f"[{i+1}/{len(symbols)}] {symbol} 처리 중...")
            df = self.get_spot_historical_klines(symbol, '1d','2017-01-01')
            if df is not None:
                df = df[['timestamp', 'close']].copy()
                df = df.rename(columns={'close': symbol})
                if merged_df.empty:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on='timestamp', how='outer')
            else:
                # 데이터가 없을 경우 NaN으로 처리
                if merged_df.empty:
                    merged_df = pd.DataFrame({'timestamp': pd.date_range(start='2017-01-01', end=pd.Timestamp.now(), freq='D')})
                merged_df[symbol] = np.nan  # NaN으로 추가
            time.sleep(2 + np.random.rand())  # API 호출 제한 방지

        merged_df = merged_df.sort_values('timestamp').reset_index(drop=True)
        merged_df.set_index('timestamp', inplace=True)
        self.spot = merged_df.copy()
        return merged_df
    

In [5]:
#심볼 확인
spots = Spot()
spots_symbols = spots.get_all_spot_symbols()
print(spots_symbols)

['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT', 'XRPUSDT', 'EOSUSDT', 'TUSDUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT', 'ICXUSDT', 'NULSUSDT', 'VETUSDT', 'USDCUSDT', 'LINKUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT', 'FETUSDT', 'BATUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'THETAUSDT', 'ENJUSDT', 'ATOMUSDT', 'TFUELUSDT', 'ONEUSDT', 'ALGOUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT', 'WINUSDT', 'COSUSDT', 'MTLUSDT', 'DENTUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT', 'BANDUSDT', 'XTZUSDT', 'RVNUSDT', 'HBARUSDT', 'NKNUSDT', 'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'CTXCUSDT', 'BCHUSDT', 'TROYUSDT', 'FTTUSDT', 'EURUSDT', 'OGNUSDT', 'LSKUSDT', 'BNTUSDT', 'LTOUSDT', 'MBLUSDT', 'COTIUSDT', 'STPTUSDT', 'DATAUSDT', 'SOLUSDT', 'CTSIUSDT', 'HIVEUSDT', 'CHRUSDT', 'ARDRUSDT', 'MDTUSDT', 'KNCUSDT', 'LRCUSDT', 'COMPUSDT', 'SCUSDT', 'ZENUSDT', 'SNXUSDT', 'VTHOUSDT', 'DGBUSDT', 'SXPUSDT', 'MKRUSDT', 'DCRUSDT', 'STORJUSD

In [6]:
#데이터 가져오기
print(Spot.get_spot_historical_klines.__doc__)



        특정 기간의 현물 캔들스틱 데이터 가져오기
        :param symbol: 거래쌍 또는 'all'
        :param interval: 시간간격
        :param start_str: 시작 날짜 (YYYY-MM-DD)
        :param end_str: 종료 날짜 (YYYY-MM-DD)
        :return: Dictionary of DataFrames (symbol을 'all'로 지정한 경우) 또는 단일 DataFrame
        


In [7]:
#적용 예시1
df = spots.get_spot_historical_klines('BTCUSDT', '1d','2017-01-01')
df

,timestamp,open,high,low,close,volume
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...,...
2787,2025-04-04,83213.09,84720.00,81659.00,83889.87,32915.539760
2788,2025-04-05,83889.87,84266.00,82379.95,83537.99,9360.404680
2789,2025-04-06,83537.99,83817.63,77153.83,78430.00,27942.714360
2790,2025-04-07,78430.00,81243.58,74508.00,79163.24,78387.530890


In [8]:
#적용 예시 2
df = spots.get_spot_historical_klines('ETHUSDT', '1d','2017-01-01')
df

,timestamp,open,high,low,close,volume
0,2017-08-17,301.13,312.18,298.00,302.00,7.030710e+03
1,2017-08-18,302.00,311.79,283.94,293.96,9.537846e+03
2,2017-08-19,293.31,299.90,278.00,290.91,2.146198e+03
3,2017-08-20,289.41,300.53,282.85,299.10,2.510139e+03
4,2017-08-21,299.10,346.52,294.60,323.29,5.219445e+03
...,...,...,...,...,...,...
2787,2025-04-04,1817.23,1835.68,1758.72,1816.87,5.727345e+05
2788,2025-04-05,1816.88,1827.29,1764.39,1806.01,2.017163e+05
2789,2025-04-06,1806.02,1817.00,1537.50,1580.76,9.798471e+05
2790,2025-04-07,1580.77,1639.00,1411.01,1553.04,2.168796e+06


In [9]:
#전 bitcoin 현물 데이터 수집
spot_df = spots.fetch_and_merge_close_prices()

[1/402] BTCUSDT 처리 중...
[2/402] ETHUSDT 처리 중...
[3/402] BNBUSDT 처리 중...
[4/402] NEOUSDT 처리 중...
[5/402] LTCUSDT 처리 중...
[6/402] QTUMUSDT 처리 중...
[7/402] ADAUSDT 처리 중...
[8/402] XRPUSDT 처리 중...
[9/402] EOSUSDT 처리 중...
[10/402] TUSDUSDT 처리 중...
[11/402] IOTAUSDT 처리 중...
[12/402] XLMUSDT 처리 중...
[13/402] ONTUSDT 처리 중...
[14/402] TRXUSDT 처리 중...
[15/402] ETCUSDT 처리 중...
[16/402] ICXUSDT 처리 중...
[17/402] NULSUSDT 처리 중...
[18/402] VETUSDT 처리 중...
[19/402] USDCUSDT 처리 중...
[20/402] LINKUSDT 처리 중...
[21/402] ONGUSDT 처리 중...
[22/402] HOTUSDT 처리 중...
[23/402] ZILUSDT 처리 중...
[24/402] ZRXUSDT 처리 중...
[25/402] FETUSDT 처리 중...
[26/402] BATUSDT 처리 중...
[27/402] ZECUSDT 처리 중...
[28/402] IOSTUSDT 처리 중...
[29/402] CELRUSDT 처리 중...
[30/402] DASHUSDT 처리 중...
[31/402] THETAUSDT 처리 중...
[32/402] ENJUSDT 처리 중...
[33/402] ATOMUSDT 처리 중...
[34/402] TFUELUSDT 처리 중...
[35/402] ONEUSDT 처리 중...
[36/402] ALGOUSDT 처리 중...
[37/402] DOGEUSDT 처리 중...
[38/402] DUSKUSDT 처리 중...
[39/402] ANKRUSDT 처리 중...
[40/402] WINUSDT

In [10]:
#결과
spot_df

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,QTUMUSDT,ADAUSDT,XRPUSDT,EOSUSDT,TUSDUSDT,...,BMTUSDT,FORMUSDT,XUSDUSDT,NILUSDT,PARTIUSDT,MUBARAKUSDT,TUTUSDT,BROCCOLI714USDT,BANANAS31USDT,GUNUSDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-08-17,4285.08,302.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-18,4108.37,293.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-19,4139.98,290.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-20,4086.29,299.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-21,4016.00,323.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-04,83889.87,1816.87,597.71,4.84,84.41,1.884,0.6603,2.1302,0.7824,0.9990,...,0.0916,2.1729,1.0001,0.427,0.1884,0.0387,0.02257,0.02855,0.005586,0.04162
2025-04-05,83537.99,1806.01,593.13,4.76,82.67,1.878,0.6547,2.1442,0.7509,0.9990,...,0.0961,2.1758,1.0002,0.422,0.1713,0.0381,0.02392,0.03244,0.005789,0.05600
2025-04-06,78430.00,1580.76,555.54,4.73,70.62,1.726,0.5737,1.9218,0.7168,0.9984,...,0.0875,1.9181,1.0001,0.443,0.1673,0.0397,0.02479,0.02852,0.005950,0.05703


In [11]:
#parquet or execl 변환
spot_df.to_parquet("Spot_17_25.parquet", engine='pyarrow')
spot_df.to_excel('Spot_17_25.xlsx',index = True )